In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import LeaveOneOut
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.datasets import load_digits

# Generative Model - Using Kernel Density Estimation

What is KDE? . . .

Why use KDE? . . .